# Fermentation data import and export

## 1. Imports and directory definitions

In [1]:
import pandas as pd # pandas for tabular data manipulation
import numpy as np  # numpy for numerical operations
import glob
import os
from pandas import ExcelWriter
import openpyxl

data_folder = '../../../data/'
fieldlist_path = '../fieldlist/fieldlist_v1.xlsx'
od_cal_file = '../2021-10-04_OD_calibration_file.xlsx'

## 2. Experient name definition

In [2]:
# "../../../data/LIMS_data/DDB_PD_036_AMBR_LIMS.xlsx"
experiment = 'DDB_PD_036_AMBR'

## 3. Import data

### 3.1 Main cultures metadata

In [3]:
# get the path to experiment in the LIMS file folder
experiment_file = data_folder+'LIMS_data/'+experiment+'_LIMS.xlsx'
# import the data from the main culture sheet and drop any column that has only NaNs
meta_data = pd.read_excel(experiment_file, sheet_name='main culture').dropna(how='all', axis=1)
    
# cultures.index = range(0,len(cultures.index)) # updating the index of the dataframe
meta_data.rename(columns={'Sample ID': 'Parent Culture'}, inplace=True)
meta_data.sort_values('Reactor/Plate/Flask Number',inplace=True)
meta_data = meta_data.reset_index().drop('index',axis=1)
meta_data

,Parent Culture,Base Medium,Comments,Container ID (calculated),Container Type,Control?,Culture Type,Experiment,Experiment Annotation,growth_rate,...,pH Control Base Solution,DO Control Setpoint (%),DO Control Cascade Level 1,DO Control Cascade Level 2,DO Control Cascade Level 3,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type
0,DDB_PD_036_AMBR_DDB35-001_R13_Main_,DDB_BM_002,Control Medium,R13,AMBR 250,No,Main,DDB_PD_036_AMBR,DDB Zinc,0.11,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2"
1,DDB_PD_036_AMBR_DDB35-001_R14_Main_,DDB_BM_002,Control Medium,R14,AMBR 250,No,Main,DDB_PD_036_AMBR,DDB Zinc,0.11,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2"
2,DDB_PD_036_AMBR_SDT107-001_R15_Main_,DDB_BM_002,Control Medium,R15,AMBR 250,No,Main,DDB_PD_036_AMBR,DDB Zinc,0.11,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2"
3,DDB_PD_036_AMBR_SDT107-001_R16_Main_,DDB_BM_002,Control Medium,R16,AMBR 250,No,Main,DDB_PD_036_AMBR,DDB Zinc,0.11,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2"
4,DDB_PD_036_AMBR_DDB35-001_R17_Main_,DDB_BM_002,2x Zn concentration (Batch medium) and 20x Zn ...,R17,AMBR 250,No,Main,DDB_PD_036_AMBR,DDB Zinc,0.11,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2"
5,DDB_PD_036_AMBR_DDB35-001_R18_Main_,DDB_BM_002,2x Zn concentration (Batch medium) and 20x Zn ...,R18,AMBR 250,No,Main,DDB_PD_036_AMBR,DDB Zinc,0.11,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2"
6,DDB_PD_036_AMBR_SDT107-001_R19_Main_,DDB_BM_002,2x Zn concentration (Batch medium) and 20x Zn ...,R19,AMBR 250,No,Main,DDB_PD_036_AMBR,DDB Zinc,0.11,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2"
7,DDB_PD_036_AMBR_SDT107-001_R20_Main_,DDB_BM_002,2x Zn concentration (Batch medium) and 20x Zn ...,R20,AMBR 250,No,Main,DDB_PD_036_AMBR,DDB Zinc,0.11,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2"
8,DDB_PD_036_AMBR_DDB35-001_R21_Main_,DDB_BM_002,0.6x Zn concentration (Batch medium) and 1/20x...,R21,AMBR 250,No,Main,DDB_PD_036_AMBR,DDB Zinc,0.11,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2"
9,DDB_PD_036_AMBR_DDB35-001_R22_Main_,DDB_BM_002,0.6x Zn concentration (Batch medium) and 1/20x...,R22,AMBR 250,No,Main,DDB_PD_036_AMBR,DDB Zinc,0.11,...,Ammonium hydroxide 15%,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2"


### 3.1 Online data

In [4]:
online_data_dir = 'online_data_interpolated/'
csv_prefix = '_online_stacked_2m.csv'

In [5]:
# import fieldlist
fieldlist = pd.read_excel(fieldlist_path)
# initialize an empty DataFrame with columns from the variables specified in the fieldlist
online_data = pd.DataFrame(columns=list(fieldlist.new_variable_name))

In [6]:
ambr_df = pd.read_csv(data_folder+ online_data_dir + str(experiment) + csv_prefix)

for column in list(online_data.columns): # iterate over the columns of the temporary dataframe. The columns are the variable names of the online data.
    # match the column name (new_variable_name) to the name given by ambr250. Then assign the ambr250 online data to the matching column in the temp_df
    online_data[column] = ambr_df[fieldlist[fieldlist.new_variable_name == column].AMBR_name.iloc[0]]

# adding a column with the experiment name
online_data['Experiment'] = experiment

# Converting the bioreactor naming format in the temporary ambr df
for i in range(1,10):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R0'+str(i))
for i in range(10,25):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R'+str(i))

# adding a column with the Reactor Number
online_data['Reactor/Plate/Flask Number'] = ambr_df['Batch ID'] # adding a column with the reactor ID
    
# dropping some unnecessary rows from the online data DataFrame:
online_data.dropna(axis=0, subset=['Inlet_air_O2'],inplace=True)
    
# updating index
online_data.index = range(0,len(online_data.index)) # updating the index of the dataframe
    
# Merging online data with cultures metadata
online_data = pd.merge(left=online_data,right=meta_data)
    
# phase annotation
feed_vol_list = online_data['Feed_volume'].tolist()
phase_list = ['feed' if i > 0.0 else 'batch' for i in feed_vol_list]
online_data['Phase'] = phase_list
online_data.head()

,Time,DO,pH,Stirrer_speed,Temperature,Pressure,Air_flow,Oxygen_flow,Inlet_air_O2,Offgas_O2,...,DO Control Setpoint (%),DO Control Cascade Level 1,DO Control Cascade Level 2,DO Control Cascade Level 3,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type,Phase
0,0.000000,96.404177,6.715000,1002.999140,29.899997,2.415055,99.944011,0.0,20.95,20.674000,...,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2",batch
1,0.033333,95.086025,6.713327,1001.163729,29.958362,2.554151,100.190860,0.0,20.95,20.677000,...,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2",batch
2,0.066667,93.908675,6.714940,1000.624083,29.990000,2.529370,100.087630,0.0,20.95,20.673188,...,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2",batch
3,0.100000,93.195827,6.716083,1000.854304,30.010000,2.427948,99.977976,0.0,20.95,20.671229,...,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2",batch
4,0.133333,92.900351,6.714545,998.545451,30.020000,2.525727,99.957000,0.0,20.95,20.669091,...,40,Stirring,Aeration,Oxygen,0.1,0.22,1000,4000,"Air, O2",batch


### 3.2 Offline data

#### 3.2.1 Timepoint data

In [7]:
timepoint_cols = ['Entity','Experiment','Parent Culture','Reactor/Plate Number','Timepoint (#)','Timepoint (h)','Volume','OD']
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experimentin the LIMS file folder

timepoint_data = pd.read_excel(file, sheet_name='timepoint samples')[timepoint_cols] # importing timepoint data from one experiment into a temporary DFF

# renaming some of the columns in the dataframe and reindexing
timepoint_data.rename(columns={"Entity": "Timepoint Sample", "Reactor/Plate Number": "Reactor/Plate/Flask Number", "Volume": "Sample volume (mL)"},inplace=True)
timepoint_data = timepoint_data.reset_index().drop('index',axis=1)
timepoint_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD
0,DDB_PD_036_AMBR_R13__S00,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,R13,S00,0.000000,1.0,0.282
1,DDB_PD_036_AMBR_R13__S01,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,R13,S01,3.018930,1.0,0.656
2,DDB_PD_036_AMBR_R13__S02,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,R13,S02,6.168409,2.5,1.996
3,DDB_PD_036_AMBR_R13__S03,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,R13,S03,9.017390,1.0,6.590
4,DDB_PD_036_AMBR_R13__S04,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,R13,S04,12.020640,1.0,22.300


#### 3.2.2 Analytical data

In [8]:
analytical_cols = ['Name', 'Analysis Type', 'Cellular Compartment', 'Dilution Factor','Experiment', 'Parent Culture', 'Sample Prep Date', 'Timepoint (h)', 'Timepoint Sample', 'compound_name', 'concentration','concentration_unit']
    
analytical_data = pd.DataFrame(columns=analytical_cols) # initializing an empty DF for timepoint_data
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experiment in the LIMS file folder

analytical_data = pd.read_excel(file, sheet_name='analytical samples')[analytical_cols] # importing timepoint data from one experiment into a temporary DF

# renaming one of the columns in the dataframe and reindexing
analytical_data.rename(columns={"Name": "Analytical Sample"},inplace=True)
analytical_data = analytical_data.reset_index().drop('index',axis=1)

# correcting for the dilution factor, then dropping the dilution factor
analytical_data['concentration'] = analytical_data['concentration'] * analytical_data['Dilution Factor']
analytical_data.drop(['Dilution Factor'], axis=1, inplace=True)
# removing any analytical data not obtained from a main culture sample
analytical_data = analytical_data[analytical_data['Parent Culture'].notnull()]
analytical_data.head()

,Analytical Sample,Analysis Type,Cellular Compartment,Experiment,Parent Culture,Sample Prep Date,Timepoint (h),Timepoint Sample,compound_name,concentration,concentration_unit
0,DDB_PD_036_AMBR_R13__S08_SOA_#1,SOA,Extracellular Region,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,2021-04-26,23.06692,DDB_PD_036_AMBR_R13__S08,succinic acid,0.0,g/L
1,DDB_PD_036_AMBR_R13__S08_SOA_#1,SOA,Extracellular Region,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,2021-04-26,23.06692,DDB_PD_036_AMBR_R13__S08,ethanol,0.0,g/L
2,DDB_PD_036_AMBR_R13__S08_SOA_#1,SOA,Extracellular Region,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,2021-04-26,23.06692,DDB_PD_036_AMBR_R13__S08,citric acid,0.0,g/L
3,DDB_PD_036_AMBR_R13__S08_SOA_#1,SOA,Extracellular Region,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,2021-04-26,23.06692,DDB_PD_036_AMBR_R13__S08,D-glucose,0.0,g/L
4,DDB_PD_036_AMBR_R13__S08_SOA_#1,SOA,Extracellular Region,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,2021-04-26,23.06692,DDB_PD_036_AMBR_R13__S08,formic acid,0.0,g/L


##### 3.2.2.1 Pivot analytical data

In [9]:
analytical_data['compound_name_and_unit'] = analytical_data['compound_name'] + ' [' + analytical_data['concentration_unit'] + ']'

analytical_data_pivot = analytical_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
analytical_data_pivot['Timepoint Sample'] = analytical_data_pivot.index

analytical_data_pivot.index.name = None
analytical_data_pivot.columns.name = None
analytical_data_pivot = analytical_data_pivot.reset_index().drop('index',axis=1)

analytical_data_pivot.head()

,D-glucose [g/L],acetic acid [g/L],citric acid [g/L],ethanol [g/L],formic acid [g/L],lactate [g/L],pyruvic acid [g/L],succinic acid [g/L],Timepoint Sample
0,21.439751,0.233741,0.0,0.0,0.0,0.000000,0.0,0.0,DDB_PD_036_AMBR_R13__S00
1,23.638231,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DDB_PD_036_AMBR_R13__S01
2,24.301256,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,DDB_PD_036_AMBR_R13__S02
3,18.325940,0.604094,0.0,0.0,0.0,0.285775,0.0,0.0,DDB_PD_036_AMBR_R13__S03
4,8.215383,1.243770,0.0,0.0,0.0,0.769308,0.0,0.0,DDB_PD_036_AMBR_R13__S04


#### 3.2.3 Trace_metal_data

In [10]:
trace_metal_data = pd.read_excel(data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx', sheet_name='trace metal export')
trace_metal_data.rename(columns={'Name': 'Analytical Sample'}, inplace=True)

# timepoint_samples_temp = []
# for i in df_tm['Analytical Sample'].to_list():
#     timepoint_samples_temp.append(i.split('_TM')[0])

# df_tm['Timepoint Sample'] = timepoint_samples_temp
trace_metal_data['Timepoint Sample'] = trace_metal_data['Analytical Sample'].apply(lambda x: pd.Series(str(x).split("_TM")[0]))
trace_metal_data.head()

,Analytical Sample,compound_name,concentration,concentration_unit,Timepoint Sample
0,DDB_PD_036_AMBR_R14__S09,v,0,µg/L,DDB_PD_036_AMBR_R14__S09
1,DDB_PD_036_AMBR_R14__S09,cr,0,µg/L,DDB_PD_036_AMBR_R14__S09
2,DDB_PD_036_AMBR_R14__S09,mn,101.5,µg/L,DDB_PD_036_AMBR_R14__S09
3,DDB_PD_036_AMBR_R14__S09,fe,78.3,µg/L,DDB_PD_036_AMBR_R14__S09
4,DDB_PD_036_AMBR_R14__S09,co,151.3,µg/L,DDB_PD_036_AMBR_R14__S09


##### 3.2.2.1 Pivot trace metal data

In [11]:
trace_metal_data['compound_name_and_unit'] = trace_metal_data['compound_name'] + ' [' + trace_metal_data['concentration_unit'] + ']'

trace_metal_data_pivot = trace_metal_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
trace_metal_data_pivot['Timepoint Sample'] = trace_metal_data_pivot.index

trace_metal_data_pivot.index.name = None
trace_metal_data_pivot.columns.name = None
trace_metal_data_pivot = trace_metal_data_pivot.reset_index().drop('index',axis=1)

trace_metal_data_pivot.head()

,as [µg/L],cd_no_gas [µg/L],co [µg/L],cr [µg/L],cu [µg/L],fe [µg/L],mn [µg/L],mo [µg/L],ni [µg/L],pb_no_gas [µg/L],v [µg/L],zn [µg/L],Timepoint Sample
0,0,0,75.2,0,150.5,4975.1,32.3,166,0,2.9,0,147.8,DDB_PD_036_AMBR_R14__S00
1,0,0,79.5,0,161.1,4787.8,35.2,159.1,0,0,0,169.8,DDB_PD_036_AMBR_R14__S01
2,0,0,79.6,0,194,5227.1,70.9,169.1,0,0,0,183.3,DDB_PD_036_AMBR_R14__S02
3,0,0,80,0,144.5,5134.9,50.6,161.2,0,0,0,128.4,DDB_PD_036_AMBR_R14__S03
4,0,0,143.6,0,141.3,4940.6,90.3,159.6,0,0,0,78,DDB_PD_036_AMBR_R14__S04


#### 3.2.4 Custom data

In [12]:
try:
    custom_data = pd.read_excel(data_folder+'/custom_data_import_files/'+experiment+'_custom.xlsx')
    custom_data.head()
except:
    custom_data = pd.DataFrame(columns=['Timepoint Sample'])
    
custom_data

,Timepoint Sample


#### 3.2.5 Merging all offline data

In [13]:
ac_merge = pd.merge(left=analytical_data_pivot, right=trace_metal_data_pivot, how='left')
custom_merge = pd.merge(left=ac_merge, right=custom_data, how='left')
offline_data = pd.merge(left=timepoint_data, right=custom_merge, how='left')
offline_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD,D-glucose [g/L],acetic acid [g/L],...,co [µg/L],cr [µg/L],cu [µg/L],fe [µg/L],mn [µg/L],mo [µg/L],ni [µg/L],pb_no_gas [µg/L],v [µg/L],zn [µg/L]
0,DDB_PD_036_AMBR_R13__S00,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,R13,S00,0.000000,1.0,0.282,21.439751,0.233741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DDB_PD_036_AMBR_R13__S01,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,R13,S01,3.018930,1.0,0.656,23.638231,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DDB_PD_036_AMBR_R13__S02,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,R13,S02,6.168409,2.5,1.996,24.301256,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DDB_PD_036_AMBR_R13__S03,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,R13,S03,9.017390,1.0,6.590,18.325940,0.604094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DDB_PD_036_AMBR_R13__S04,DDB_PD_036_AMBR,DDB_PD_036_AMBR_DDB35-001_R13_Main_,R13,S04,12.020640,1.0,22.300,8.215383,1.243770,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 4. Export data

### 4.2 Separate csv files

In [14]:
meta_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_meta_data_export.csv',index=False)
offline_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_offline_data_export.csv',index=False)
online_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_online_data_export.csv',index=False)